In [1]:
import numpy as np
import pandas as pd

# Load dataset from CSV
df = pd.read_csv("student_dataset.csv")

# Extract features and labels
X = df.drop(columns=["Final_Score"]).values  # All columns except target
y = df["Final_Score"].values.reshape(-1, 1)  # Target column

# Normalize features
X = X / np.amax(X, axis=0)
y = y / 100  # Normalize target

class Neural_Network(object):
    def __init__(self):
        self.inputSize = X.shape[1]
        self.outputSize = 1
        self.hiddenSize = 8  # Increased hidden layer size for better learning
        
        # Weights
        self.W1 = np.random.randn(self.inputSize, self.hiddenSize)  # Weight matrix from input to hidden layer
        self.W2 = np.random.randn(self.hiddenSize, self.outputSize) # Weight matrix from hidden to output layer

    def forward(self, X):
        self.z = np.dot(X, self.W1)        # Dot product of input and first set of weights
        self.z2 = self.sigmoid(self.z)     # Activation function
        self.z3 = np.dot(self.z2, self.W2) # Dot product of hidden layer and second set of weights
        o = self.sigmoid(self.z3)          # Final activation
        return o

    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)  # Derivative of sigmoid
    
    def backward(self, X, y, o):
        self.o_error = y - o  # Error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o)  # Applying derivative of sigmoid to output error
        
        self.z2_error = self.o_delta.dot(self.W2.T)  # Error in hidden layer
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)  # Applying derivative of sigmoid
        
        self.W1 += X.T.dot(self.z2_delta)  # Adjust input -> hidden weights
        self.W2 += self.z2.T.dot(self.o_delta)  # Adjust hidden -> output weights

    def train(self, X, y):
        o = self.forward(X)
        self.backward(X, y, o)

NN = Neural_Network()
print("\nInput: \n", X)
print("\nActual Output: \n", y)
print("\nPredicted Output: \n", NN.forward(X))
print("\nLoss: \n", np.mean(np.square(y - NN.forward(X))))  # Mean squared loss

NN.train(X, y)



Input: 
 [[0.56496031 0.73607334 0.59274022 ... 0.31940835 0.4644783  0.28763054]
 [0.97027829 0.59423112 0.77117653 ... 0.81909783 0.17422815 0.55832177]
 [0.81641632 0.39593447 0.93674751 ... 0.19642156 0.33796344 0.30454901]
 ...
 [0.35616825 0.24477677 0.73196159 ... 0.81807558 0.07418167 0.86345524]
 [0.98693668 0.95669402 0.69009518 ... 0.74846349 0.99426307 0.16303189]
 [0.99524877 0.51538237 0.93194001 ... 0.1776084  0.40848212 0.81305817]]

Actual Output: 
 [[0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0

In [2]:
import csv
import random
import math

# Load dataset manually
def load_csv(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)
    return [[float(value) for value in row] for row in data[1:]]  # Skip header

data = load_csv("student_dataset.csv")
X = [row[:-1] for row in data]  # Features
y = [[row[-1] / 100] for row in data]  # Normalize target

# Initialize weights
def initialize_weights(input_size, hidden_size, output_size):
    W1 = [[random.uniform(-1, 1) for _ in range(hidden_size)] for _ in range(input_size)]
    W2 = [[random.uniform(-1, 1) for _ in range(output_size)] for _ in range(hidden_size)]
    return W1, W2

# Activation function
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Forward pass
def forward_pass(X, W1, W2):
    z = [[sum(i * w for i, w in zip(row, col)) for col in zip(*W1)] for row in X]
    z2 = [[sigmoid(val) for val in row] for row in z]
    z3 = [[sum(i * w for i, w in zip(row, col)) for col in zip(*W2)] for row in z2]
    o = [[sigmoid(val) for val in row] for row in z3]
    return z, z2, z3, o

# Backpropagation
def backward_pass(X, y, W1, W2, z, z2, z3, o, learning_rate=0.01):
    o_error = [[yi - oi for yi, oi in zip(y_row, o_row)] for y_row, o_row in zip(y, o)]
    o_delta = [[err * sigmoid_derivative(oi) for err, oi in zip(err_row, o_row)] for err_row, o_row in zip(o_error, o)]
    
    z2_error = [[sum(d * w for d, w in zip(delta_row, col)) for col in zip(*W2)] for delta_row in o_delta]
    z2_delta = [[err * sigmoid_derivative(zi) for err, zi in zip(err_row, z2_row)] for err_row, z2_row in zip(z2_error, z2)]
    
    for i in range(len(W1)):
        for j in range(len(W1[i])):
            W1[i][j] += learning_rate * sum(X[k][i] * z2_delta[k][j] for k in range(len(X)))
    
    for i in range(len(W2)):
        for j in range(len(W2[i])):
            W2[i][j] += learning_rate * sum(z2[k][i] * o_delta[k][j] for k in range(len(z2)))
    
    return W1, W2

# Training function
def train(X, y, hidden_size=8, epochs=100, learning_rate=0.01):
    input_size, output_size = len(X[0]), 1
    W1, W2 = initialize_weights(input_size, hidden_size, output_size)
    
    for epoch in range(epochs):
        z, z2, z3, o = forward_pass(X, W1, W2)
        W1, W2 = backward_pass(X, y, W1, W2, z, z2, z3, o, learning_rate)
    
    return W1, W2, o

# Train the model
W1, W2, predictions = train(X, y)

# Print results
print("\nInput: \n", X)
print("\nActual Output: \n", y)
print("\nPredicted Output: \n", predictions)
print("\nLoss: \n", sum(sum((yi - oi) ** 2 for yi, oi in zip(y_row, o_row)) for y_row, o_row in zip(y, predictions)) / len(y))


IndexError: list index out of range

In [3]:
import csv
import random
import math

# Load dataset manually
def load_csv(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)
    return [[float(value) for value in row] for row in data[1:]]  # Skip header

data = load_csv("student_dataset.csv")
X = [row[:-1] for row in data]  # Features
y = [[row[-1] / 100] for row in data]  # Normalize target

# Initialize weights
def initialize_weights(input_size, hidden_size, output_size):
    W1 = [[random.uniform(-1, 1) for _ in range(hidden_size)] for _ in range(input_size)]
    W2 = [[random.uniform(-1, 1) for _ in range(output_size)] for _ in range(hidden_size)]
    return W1, W2

# Activation function
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Forward pass
def forward_pass(X, W1, W2):
    z = [[sum(i * w for i, w in zip(row, col)) for col in zip(*W1)] for row in X]
    z2 = [[sigmoid(val) for val in row] for row in z]
    z3 = [[sum(i * w for i, w in zip(row, col)) for col in zip(*W2)] for row in z2]
    o = [[sigmoid(val) for val in row] for row in z3]
    return z, z2, z3, o

# Backpropagation
def backward_pass(X, y, W1, W2, z, z2, z3, o, learning_rate=0.01):
    o_error = [[yi - oi for yi, oi in zip(y_row, o_row)] for y_row, o_row in zip(y, o)]
    o_delta = [[err * sigmoid_derivative(oi) for err, oi in zip(err_row, o_row)] for err_row, o_row in zip(o_error, o)]
    
    z2_error = [[sum(d * w for d, w in zip(delta_row, col)) for col in zip(*W2)] for delta_row in o_delta]
    z2_delta = [[err * sigmoid_derivative(zi) for err, zi in zip(err_row, z2_row)] for err_row, z2_row in zip(z2_error, z2)]
    
    for i in range(len(W1)):
        for j in range(len(W1[i])):
            W1[i][j] += learning_rate * sum(X[k][i] * z2_delta[k][j] for k in range(len(X)))
    
    for i in range(len(W2)):
        for j in range(len(W2[i])):
            W2[i][j] += learning_rate * sum(z2[k][i] * o_delta[k][j] for k in range(len(z2)))
    
    return W1, W2

# Training function
def train(X, y, hidden_size=8, epochs=100, learning_rate=0.01):
    input_size, output_size = len(X[0]), 1
    W1, W2 = initialize_weights(input_size, hidden_size, output_size)
    
    for epoch in range(epochs):
        z, z2, z3, o = forward_pass(X, W1, W2)
        W1, W2 = backward_pass(X, y, W1, W2, z, z2, z3, o, learning_rate)
    
    return W1, W2, o

# Train the model
W1, W2, predictions = train(X, y)

# Print results
print("\nInput: \n", X)
print("\nActual Output: \n", y)
print("\nPredicted Output: \n", predictions)
print("\nLoss: \n", sum(sum((yi - oi) ** 2 for yi, oi in zip(y_row, o_row)) for y_row, o_row in zip(y, predictions)) / len(y))

IndexError: list index out of range

In [1]:
import numpy as np
import pandas as pd

# Load dataset from CSV
df = pd.read_csv("student_dataset.csv")

# Extract features and labels
X = df.drop(columns=["Final_Score"]).values  # All columns except target
y = df["Final_Score"].values.reshape(-1, 1)  # Target column

# Normalize features
X = X / np.amax(X, axis=0)
y = y / 100  # Normalize target

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Weights initialization
        self.W1 = np.random.randn(self.input_size, self.hidden_size) * 0.01
        self.W2 = np.random.randn(self.hidden_size, self.output_size) * 0.01
    
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoid_derivative(self, s):
        return s * (1 - s)
    
    def forward(self, X):
        self.z1 = np.dot(X, self.W1)
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2)
        self.output = self.sigmoid(self.z2)
        return self.output
    
    def backward(self, X, y):
        output_error = y - self.output  # Error in output
        output_delta = output_error * self.sigmoid_derivative(self.output)  # Applying derivative of sigmoid
        
        hidden_error = output_delta.dot(self.W2.T)  # Error in hidden layer
        hidden_delta = hidden_error * self.sigmoid_derivative(self.a1)  # Applying derivative of sigmoid
        
        # Weight updates
        self.W2 += self.a1.T.dot(output_delta) * self.learning_rate
        self.W1 += X.T.dot(hidden_delta) * self.learning_rate
    
    def train(self, X, y, epochs=10000):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)
            if epoch % 1000 == 0:
                loss = np.mean(np.square(y - self.output))
                print(f"Epoch {epoch}, Loss: {loss}")

# Initialize and train the neural network
input_size = X.shape[1]
hidden_size = 8  # You can adjust this
output_size = 1
learning_rate = 0.1

NN = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
print("Initial Loss:", np.mean(np.square(y - NN.forward(X))))

NN.train(X, y)

print("\nFinal Predicted Output:\n", NN.forward(X))


Initial Loss: 2.694973452718413e-05
Epoch 0, Loss: 2.694973452718413e-05
Epoch 1000, Loss: 2.504571447809962e-12
Epoch 2000, Loss: 2.4882017140148408e-12
Epoch 3000, Loss: 2.471974271400901e-12
Epoch 4000, Loss: 2.4558875301936063e-12
Epoch 5000, Loss: 2.4399399224909473e-12
Epoch 6000, Loss: 2.42412990182504e-12
Epoch 7000, Loss: 2.408455942908207e-12
Epoch 8000, Loss: 2.3929165412137383e-12
Epoch 9000, Loss: 2.3775102127126853e-12

Final Predicted Output:
 [[0.50000146]
 [0.49999669]
 [0.50000103]
 [0.49999809]
 [0.4999989 ]
 [0.49999918]
 [0.50000126]
 [0.49999945]
 [0.50000029]
 [0.50000098]
 [0.5000002 ]
 [0.49999846]
 [0.49999917]
 [0.50000044]
 [0.49999928]
 [0.49999808]
 [0.49999983]
 [0.50000069]
 [0.49999954]
 [0.5000009 ]
 [0.50000216]
 [0.49999613]
 [0.500002  ]
 [0.49999982]
 [0.50000111]
 [0.50000086]
 [0.50000098]
 [0.50000061]
 [0.49999878]
 [0.50000241]
 [0.50000065]
 [0.4999978 ]
 [0.50000136]
 [0.50000278]
 [0.49999975]
 [0.49999858]
 [0.49999869]
 [0.50000113]
 [0.5

In [2]:
import numpy as np
import pandas as pd

# Load dataset from CSV
df = pd.read_csv("student_dataset.csv")

# Extract features and labels
X = df.drop(columns=["Final_Score"]).values  # All columns except target
y = df["Final_Score"].values.reshape(-1, 1)  # Target column

# Normalize features
X = X / np.amax(X, axis=0)
y = y / 100  # Normalize target

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Weights initialization
        self.W1 = np.random.randn(self.input_size, self.hidden_size) * 0.01
        self.W2 = np.random.randn(self.hidden_size, self.output_size) * 0.01
    
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoid_derivative(self, s):
        return s * (1 - s)
    
    def forward(self, X):
        self.z1 = np.dot(X, self.W1)
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2)
        self.output = self.sigmoid(self.z2)
        return self.output
    
    def backward(self, X, y):
        output_error = y - self.output  # Error in output
        output_delta = output_error * self.sigmoid_derivative(self.output)  # Applying derivative of sigmoid
        
        hidden_error = output_delta.dot(self.W2.T)  # Error in hidden layer
        hidden_delta = hidden_error * self.sigmoid_derivative(self.a1)  # Applying derivative of sigmoid
        
        # Weight updates
        self.W2 += self.a1.T.dot(output_delta) * self.learning_rate
        self.W1 += X.T.dot(hidden_delta) * self.learning_rate
    
    def train(self, X, y, epochs=10000):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)
            if epoch % 1000 == 0:
                loss = np.mean(np.square(y - self.output))
                print(f"Epoch {epoch}, Loss: {loss}")

# Initialize and train the neural network
input_size = X.shape[1]
hidden_size = 8  # You can adjust this
output_size = 1
learning_rate = 0.1

NN = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
print("Initial Loss:", np.mean(np.square(y - NN.forward(X))))

NN.train(X, y)

print("\nFinal Predicted Output:\n", NN.forward(X))


Initial Loss: 5.719681886866199e-06
Epoch 0, Loss: 5.719681886866199e-06
Epoch 1000, Loss: 3.5038713761829592e-12
Epoch 2000, Loss: 3.4947379924779472e-12
Epoch 3000, Loss: 3.485636416240578e-12
Epoch 4000, Loss: 3.4765665179750923e-12
Epoch 5000, Loss: 3.4675281687818863e-12
Epoch 6000, Loss: 3.45852124040763e-12
Epoch 7000, Loss: 3.4495456051316826e-12
Epoch 8000, Loss: 3.4406011359036678e-12
Epoch 9000, Loss: 3.4316877062651687e-12

Final Predicted Output:
 [[0.49999882]
 [0.49999895]
 [0.50000071]
 [0.50000032]
 [0.50000255]
 [0.50000228]
 [0.50000098]
 [0.49999903]
 [0.49999845]
 [0.4999991 ]
 [0.50000478]
 [0.49999989]
 [0.50000019]
 [0.50000012]
 [0.50000218]
 [0.50000135]
 [0.49999897]
 [0.49999998]
 [0.49999818]
 [0.50000116]
 [0.50000255]
 [0.49999838]
 [0.50000054]
 [0.50000124]
 [0.50000036]
 [0.49999815]
 [0.50000265]
 [0.50000087]
 [0.50000073]
 [0.50000162]
 [0.50000063]
 [0.49999885]
 [0.49999972]
 [0.49999812]
 [0.49999309]
 [0.49999915]
 [0.49999934]
 [0.5000011 ]
 [0

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np

# Load dataset from CSV
df = pd.read_csv("student_dataset.csv")

# Extract features and labels
X = df.drop(columns=["Final_Score"]).values  # All columns except target
y = df["Final_Score"].values.reshape(-1, 1)  # Target column

# Normalize features
X = X / np.amax(X, axis=0)
y = y / 100  # Normalize target

# Define the neural network model
model = Sequential([
    Dense(8, activation='sigmoid', input_shape=(X.shape[1],)),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=1000, verbose=1)

# Make predictions
predicted_output = model.predict(X)
print("\nFinal Predicted Output:\n", predicted_output)


Epoch 1/1000


F:\jupyter\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0059  
Epoch 2/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0013 
Epoch 3/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.0041e-04 
Epoch 4/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.7473e-04 
Epoch 5/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3630e-04 
Epoch 6/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4164e-04 
Epoch 7/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6204e-04 
Epoch 8/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.1926e-04 
Epoch 9/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.0715e-04 
Epoch 10/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.9620e-04 
Epoch 11/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.9305e-04 
Epoch 12/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7007e-04 
Epoch 13/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.6663e-04 
Epoch 14/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5199e-04 
Epoch 1

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6639e-05 
Epoch 78/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4692e-05 
Epoch 79/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4916e-05 
Epoch 80/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5060e-05 
Epoch 81/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3668e-05 
Epoch 82/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3838e-05 
Epoch 83/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3136e-05 
Epoch 84/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2842e-05 
Epoch 85/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3355e-05 
Epoch 86/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2046e-05 
Epoch 87/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1604e-05 
Epoch 88/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0775e-05 
Epoch 89/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1269e-05 
Epoch 90/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.05

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4385e-06 
Epoch 154/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6964e-06 
Epoch 155/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5824e-06 
Epoch 156/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4833e-06 
Epoch 157/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5022e-06 
Epoch 158/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4415e-06 
Epoch 159/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2870e-06 
Epoch 160/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4704e-06 
Epoch 161/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3907e-06 
Epoch 162/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3400e-06 
Epoch 163/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3954e-06 
Epoch 164/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2711e-06 
Epoch 165/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2652e-06 
Epoch 166/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

...

Epoch 682/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.6978e-08 
Epoch 683/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1054e-07 
Epoch 684/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.3107e-08 
Epoch 685/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.2397e-08 
Epoch 686/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0383e-07 
Epoch 687/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.0177e-08 
Epoch 688/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.7469e-08 
Epoch 689/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1978e-07 
Epoch 690/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.7736e-08 
Epoch 691/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.8078e-08 
Epoch 692/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1940e-07 
Epoch 693/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.3520e-08 
Epoch 694/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.5741e-08 
Epoch 695/1000
16/16 ━━━━━━━━━━━━━━━━━

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1296e-07 
Epoch 758/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.4795e-08 
Epoch 759/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.9098e-07 
Epoch 760/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.1409e-08 
Epoch 761/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.0840e-08 
Epoch 762/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.7206e-08 
Epoch 763/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.2137e-08 
Epoch 764/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.3196e-08 
Epoch 765/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.3964e-08 
Epoch 766/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.3362e-08 
Epoch 767/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.2687e-08 
Epoch 768/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.8387e-08 
Epoch 769/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.2157e-08 
Epoch 770/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Epoch 833/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.0155e-08 
Epoch 834/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.2945e-08 
Epoch 835/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.6491e-08 
Epoch 836/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.7899e-08 
Epoch 837/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.8198e-08 
Epoch 838/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.1725e-08 
Epoch 839/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.0336e-08 
Epoch 840/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5291e-08 
Epoch 841/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.9701e-08 
Epoch 842/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.6655e-08 
Epoch 843/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0119e-08 
Epoch 844/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.1218e-08 
Epoch 845/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.2124e-08 
Epoch 846/1000
16/16 ━━━━━━━━━━━━━━━━━

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0752e-07 
Epoch 909/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.4533e-08 
Epoch 910/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.2267e-08 
Epoch 911/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.1750e-08 
Epoch 912/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.2265e-08 
Epoch 913/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.9238e-08 
Epoch 914/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.9587e-08 
Epoch 915/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.6927e-08 
Epoch 916/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.1224e-08 
Epoch 917/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.9660e-08 
Epoch 918/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.3800e-08 
Epoch 919/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.8565e-08 
Epoch 920/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.7289e-08 
Epoch 921/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Epoch 984/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.3878e-08 
Epoch 985/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.9411e-08 
Epoch 986/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8533e-08 
Epoch 987/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.2531e-08 
Epoch 988/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.5264e-08 
Epoch 989/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.2796e-08 
Epoch 990/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.9165e-08 
Epoch 991/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.3242e-08 
Epoch 992/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.3377e-08 
Epoch 993/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5498e-08 
Epoch 994/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.7157e-08 
Epoch 995/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.4254e-08 
Epoch 996/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.9556e-08 
Epoch 997/1000
16/16 ━━━━━━━━━━━━━━━━━

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np

# Load dataset from CSV
df = pd.read_csv("student_dataset.csv")

# Extract features and labels
X = df.drop(columns=["Final_Score"]).values  # All columns except target
y = df["Final_Score"].values.reshape(-1, 1)  # Target column

# Normalize features
X = X / np.amax(X, axis=0)
y = y / 100  # Normalize target

# Define the neural network model
model = Sequential([
    Dense(8, activation='sigmoid', input_shape=(X.shape[1],)),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=1000, verbose=1)

# Make predictions
predicted_output = model.predict(X)
print("\nFinal Predicted Output:\n", predicted_output)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject